Imposta Tensorflow

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.0


Carica un dataset

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 3s 0us/step


Costruisci un modello di apprendimento automatico


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Per ogni esempio, il modello restituisce un vettore di punteggi logit o log-odds, uno per ogni classe

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.8929158 ,  0.19065458, -0.18269722, -0.46216547, -0.14830476,
         0.3079152 , -0.31729326,  0.15535757, -0.34669736, -0.1361746 ]],
      dtype=float32)

La funzione tf.nn.softmax converte questi logit in probabilità per ogni classe:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.04662973, 0.13780072, 0.09486518, 0.0717357 , 0.09818459,
        0.15494485, 0.08291871, 0.13302161, 0.08051606, 0.09938283]],
      dtype=float32)

Definire una funzione di perdita per l'allenamento utilizzando losses.SparseCategoricalCrossentropy , che accetta un vettore di logit e un indice True e restituisce una perdita scalare per ogni esempio.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Questa perdita è uguale alla probabilità logaritmica negativa della classe vera: la perdita è zero se il modello è sicuro della classe corretta.

Questo modello non addestrato fornisce probabilità vicine a casuali (1/10 per ogni classe), quindi la perdita iniziale dovrebbe essere vicina a -tf.math.log(1/10) ~= 2.3 .

In [15]:
loss_fn(y_train[:1], predictions).numpy()

1.864686

Prima di iniziare l'addestramento, configura e compila il modello utilizzando Keras Model.compile . Imposta la classe optimizer su adam , imposta la loss sulla funzione loss_fn definita in precedenza e specifica una metrica da valutare per il modello impostando il parametro metrics su accuracy .

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Allena e valuta il tuo modello
Usa il metodo Model.fit per regolare i parametri del tuo modello e ridurre al minimo la perdita:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2912 - accuracy: 0.9140
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1422 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1082 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0863 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0748 - accuracy: 0.9763


Il metodo Model.evaluate verifica le prestazioni dei modelli, solitamente su un " Set di convalida " o " Set di test ".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0734 - accuracy: 0.9790 - 515ms/epoch - 2ms/step


[0.07339996099472046, 0.9789999723434448]

Il classificatore di immagini è ora addestrato con una precisione del 98% circa su questo set di dati.

Se vuoi che il tuo modello restituisca una probabilità, puoi avvolgere il modello addestrato e allegare ad esso il softmax:



In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.0398763e-08, 2.3853708e-08, 2.0716172e-07, 9.5324482e-05,
        3.2731104e-10, 1.3070462e-07, 7.4985570e-14, 9.9989760e-01,
        4.2348293e-07, 6.2817453e-06],
       [3.3237604e-09, 2.2756845e-04, 9.9968731e-01, 8.3738509e-05,
        1.6545540e-13, 1.1086189e-06, 1.0880832e-07, 3.7761246e-13,
        9.3739978e-08, 3.6014130e-15],
       [5.4044360e-08, 9.9745387e-01, 1.0224013e-03, 8.4831363e-06,
        3.7840342e-05, 4.0284823e-07, 1.6660320e-05, 1.1493117e-03,
        3.1061246e-04, 4.6087953e-07],
       [9.9857318e-01, 3.8418756e-07, 1.3331517e-04, 2.2321052e-07,
        5.6284061e-05, 1.6886732e-05, 8.2960544e-04, 2.1501804e-05,
        8.6178353e-08, 3.6841584e-04],
       [1.7150903e-06, 3.4994549e-08, 1.2456202e-05, 8.1220250e-08,
        9.8761290e-01, 6.7728007e-07, 2.2082963e-07, 5.7422210e-05,
        9.3236958e-06, 1.2305103e-02]], dtype=float32)>